# Thomas fire

Author: Patricia Park

Repository: https://github.com/p-park6/aqi-false-color.git

## Highlights in this notebook:
- Data tidying and filtering with `pandas`
- Data statistical analysis
- Geospacial data tidying with `geopandas`
- Visualizing data through the forms of a graph and map

## About the Data
Three types of datasets were uses:
1. The first dataset covers the [Air Quality Index (AQI)](https://www.airnow.gov/aqi/aqi-basics/) found on the [US Environmental Protection Agency](https://www.epa.gov) website. Here, we will use the data provided to be able to visualize the air quality over months, and if there were any abnormal spikes that appeared during the [Thomas Fire](https://keyt.com/news/local-news/top-stories/2022/12/04/thomas-fire-5th-year-anniversary/) in 2017.

2. The second dataset includes a collection of bands (including red, green, blue, near-infrared and shortwave infrared). This data is from the [Landsat Collection 2 Surface Reflectance](https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-reflectance), collected by the Landsat 8 satellite. The data was accessed and pre-processed in the Microsoft Planetary Computer to remove data outside land and coarsen the spatial resolution ([Landsat Collection in MPC](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)). Data should be used for visualization purposes only.
- Additional information:
    -[Landsat Satellite homepage](https://www.usgs.gov/landsat-missions)


3. The third dataset include the shapefile of the fire perimeters in California during 2017. The [complete file can be accessed in the CA state geoportal](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/about).

## Final outputs:
This notebook will output a graph of the AQI over a number of months in Santa Barbara as well as a map showing the area of where the Thomas fire took place.

## Import libraries

In [1]:
import os
import numpy as np
import pandas as pd

import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
import matplotlib.patches as mpatches
from matplotlib.markers import MarkerStyle

from shapely import Point
from shapely import Polygon
from shapely import box
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

### Read in necessary data

In [ ]:
# read in daily AQI 2017 zip file from url
aqi_17 = pd.read_csv("https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip")

# read in daily AQI 2018 zip file from url
aqi_18 = pd.read_csv("https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip")

# Read in bands dataset
#create pathway
ca_bands_fp = os.path.join(os.getcwd(),'data','landsat8-2018-01-26-sb-simplified.nc')
#read data using pathway
ca_fires_bands_2017 = rioxr.open_rasterio(ca_bands_fp)

# Read in California perimeter dataset
ca_fires_perimeter_2017= gpd.read_file(os.path.join(os.getcwd(),'data', 'California_Fire_Perimeters_2017', 'California_Fire_Perimeters_2017.shp'))

## AQI data

### Prepare AQI data

### Data Exploration:

In [2]:
#look at the first 5 observations of aqi 17
aqi_17.head()

NameError: name 'aqi_17' is not defined

In [4]:
#look at the first 5 observations of aqi 18
aqi_18.head()

NameError: name 'aqi_18' is not defined

### Merge datasets together

To make data easier to access, we will merge the two datasets together

In [5]:
#join the two datasets together
aqi = pd.concat([aqi_17, aqi_18])
#print to see if the two were joined together sucessfully
aqi

NameError: name 'aqi_17' is not defined

### Reformat columns

For easier access, we will rename and reformat the column names

In [6]:
# change column names to lowercase and replace spaces with '_'
aqi.columns = aqi.columns.str.lower().str.replace(' ','_')

NameError: name 'aqi' is not defined

### Filter data

Now, we will filter our data for the appropriate columns we want to focus on. In this case, we want to look at data from Santa Barbara county and appropriate columns.

In [7]:
#filter for observations that are in Santa Barbara County
aqi_sb = aqi[aqi['county_name'] == 'Santa Barbara']

#remove not needed columns from the data frame
aqi_sb = aqi_sb.drop(columns=['state_name','county_name','state_code','county_code'])

#use column function on aqi_sb dataframe to see if columns were removed
print(aqi_sb.columns)

NameError: name 'aqi' is not defined

### Change datetime and index

In order for the graph to show the dates in a correct format, we will change the date column to the datetime format. We will also update the index to be the date column

In [9]:
# change date type to be in datetime object (can be ran only one time)
aqi_sb.date = pd.to_datetime(aqi_sb.date)

#change index to be set to date
aqi_sb = aqi_sb.set_index('date')

#check to see if the index has been updated to date
aqi_sb.index

NameError: name 'aqi_sb' is not defined

## Data Analysis and Visualization

### Create rolling mean

While it may be useful to graph every day's AQI, it is also useful to graph a rolling average to see what parts stay constant and what parts have big changes. Here, we will find the rolling average over a 5 day period.

In [10]:
#find the mean of a five day rolling average
aqi_sb.aqi.rolling('5D').mean()

#create a new column that is the 5 day rolling average mean
aqi_sb['five_day_average'] = aqi_sb.aqi.rolling('5D').mean()

NameError: name 'aqi_sb' is not defined

### Plot data

In [11]:
#Plot data
aqi_sb.plot(y = ['aqi', 'five_day_average'], #plot both aqi and five_day_average column
           title = 'AQI in Santa Barbara', #add title to graph
           xlabel = 'Year', #add x label to graph
           ylabel = 'Air Quality Index (AQI)', #add y label to graph
           color = {'aqi': 'blue', #add blue to aqi line
                    'five_day_average': 'orange' #add orange to five_day_average line
                   }
           )

NameError: name 'aqi_sb' is not defined

At around December 2017, There is a large spike in the AQI with both the daily and 5 day average. Surprisingly, there in an increase in AQI at January 2017. I assume there was a fire at the beginning of that year.

## Landsat Band Raster

### Prepare Raster data

### Data Exploration:

Let's go ahead and read in our band raster data to see what it contains

In [15]:
#print bands data
ca_fires_bands_2017

NameError: name 'ca_fires_bands_2017' is not defined

### Squeeze and drop bands

In order to map this out, we need to drop our bands. If the bands are not dropped, our raster will not be graphable as it contains more than 2 axis.

In [ ]:
#get rid of band in ca_fires_band_2017
# original dimensions and coordinates
print('original dimensions: ', ca_fires_bands_2017.dims,'\n')

# remove length 1 dimension (band)
ca_fires_bands_2017 = ca_fires_bands_2017.squeeze()
print('removed band from dimensions: ', ca_fires_bands_2017.dims,'\n')

# remove coordinates associated to band
ca_fires_bands_2017 = ca_fires_bands_2017.drop('band')
print('new dimensions: ', ca_fires_bands_2017.dims)

### Create a False Color image

Now that our raster file is ready, we want to be able to map it. Mapping a true color image will make it harder to notice the burn area of where the Thomas fire took place, so we will map a False Color image in order to make it visually easier to see where the burned area took place.

In [ ]:
#plot a false color image by plotting the swir22, nir, and red variables
ca_fires_bands_2017[['swir22', 
                     'nir08', 
                     'red']].to_array().plot.imshow(robust = True)

## California Shapefile data

### Prepare Shapefile data

### Data Exploration:

In [12]:
#look at head of fire perimeter
ca_fires_perimeter_2017.head()

#plot data to see what it looks like
ca_fires_perimeter_2017.plot()

NameError: name 'ca_fires_perimeter_2017' is not defined

### Reformat columns

To make data wrangling easier, we will be changing the column names to make it easier to interact with.

In [14]:
#change column names:
#make columns lower case
ca_fires_perimeter_2017.columns = ca_fires_perimeter_2017.columns.str.lower()
#print columns names to make sure they were changed
ca_fires_perimeter_2017.columns

NameError: name 'ca_fires_perimeter_2017' is not defined

### Change California shapefile CRS

In order to map this perimeter on top of the raster map, we need to make sure the two CRS's are the same in order for the overlap to be exact. We will go ahead and change the CRS in this step: